# Laboratorio 9

### Integrantes:

 - BRANDON JAVIER REYES MORALES
 - CARLOS ALBERTO VALLADARES GUERRA 
 - JUAN PABLO SOLIS ALBIZUREZ

### **Task 1: Teoria**

1. Diga cuál es la diferencia entre Modelos de Markov y Hidden Markov Models
- En los modelos de markov se representan sistemas donde el estado actual depende únicamente del estado anterior. Estos modelos asumen que todos los estados son observables. Por otro lado, los Hidden Markov Models suponen que los estados verdaderos del sistema son ocultos y no se pueden observar directamente. En su lugar, se observan señales o emisiones que dependen probabilísticamente de estos estados ocultos.

2. Investigue qué son los factorial HMM (Hidden Markov Models)
- Los factorial Hidden Markov Models  son una generalización de los Hidden Markov Models tradicionales. En lugar de tener una sola cadena de estados ocultos, los FHMM utilizan múltiples cadenas ocultas que evolucionan de forma paralela e independiente. Cada cadena representa una parte diferente del sistema oculto, y las observaciones se generan combinando la información de todas las cadenas. Esto permite modelar fenómenos más complejos, ya que se puede capturar la influencia simultánea de múltiples factores ocultos.

3. Especifique en sus propias palabras el algoritmo Forward-Backward para HMM
- El algoritmo Forward-Backward es una técnica utilizada en los HMM para calcular la probabilidad de que el sistema esté en un cierto estado en un momento dado, considerando toda la secuencia de observaciones. Este algoritmo tiene dos fases: la fase "Forward" calcula, hacia adelante en el tiempo, la probabilidad acumulada de haber alcanzado cada estado, dada la secuencia de observaciones hasta ese punto. La fase "Backward" hace el cálculo inverso: determina la probabilidad de observar la secuencia restante a partir de un estado específico, hacia atrás en el tiempo. 

4. En el algoritmo de Forward-Backward, ¿por qué es necesario el paso de Backward?
- El paso de Backward es importante porque complementa la información histórica (Forward) con la información futura. Si solo se usa Forward, las decisiones se basarían únicamente en el pasado y no considerarían cómo las observaciones posteriores influyen en la estimación actual. Esto es importante en aplicaciones como el reconocimiento de voz, predicciones médicas, donde las evidencias futuras mejoran significativamente la inferencia de estados ocultos.

### Task 2

In [2]:
import numpy as np
import random

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        sequence = []
        current_state = random.choices(self.states, weights=[self.initial_prob[s] for s in self.states])[0]
        
        for _ in range(length):
            obs = random.choices(self.observations, weights=[self.emission_prob[current_state][o] for o in self.observations])[0]
            sequence.append(obs)
            current_state = random.choices(self.states, weights=[self.transition_prob[current_state][s] for s in self.states])[0]
        
        return sequence

    def forward(self, obs_sequence):
        forward_probs = [{}]
        for state in self.states:
            forward_probs[0][state] = self.initial_prob[state] * self.emission_prob[state][obs_sequence[0]]

        for t in range(1, len(obs_sequence)):
            forward_probs.append({})
            for curr_state in self.states:
                prob_sum = sum(
                    forward_probs[t-1][prev_state] * self.transition_prob[prev_state][curr_state]
                    for prev_state in self.states
                )
                forward_probs[t][curr_state] = prob_sum * self.emission_prob[curr_state][obs_sequence[t]]
        
        return forward_probs

    def backward(self, obs_sequence):
        backward_probs = [{} for _ in range(len(obs_sequence))]
        for state in self.states:
            backward_probs[-1][state] = 1

        for t in reversed(range(len(obs_sequence) - 1)):
            for curr_state in self.states:
                backward_probs[t][curr_state] = sum(
                    self.transition_prob[curr_state][next_state] *
                    self.emission_prob[next_state][obs_sequence[t+1]] *
                    backward_probs[t+1][next_state]
                    for next_state in self.states
                )
        
        return backward_probs

    def compute_state_probabilities(self, obs_sequence):
        forward_probs = self.forward(obs_sequence)
        backward_probs = self.backward(obs_sequence)

        state_probs = [{} for _ in range(len(obs_sequence))]

        for t in range(len(obs_sequence)):
            total_prob = sum(
                forward_probs[t][state] * backward_probs[t][state]
                for state in self.states
            )
            for state in self.states:
                state_probs[t][state] = (forward_probs[t][state] * backward_probs[t][state]) / total_prob
        
        return state_probs


states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Rainy']

initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}

transition_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}
}

emission_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}
}

hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# --- Impresiones usando print("texto" + str(variable)) ---
obs_sequence = hmm.generate_sequence(5)
print("Secuencia Generada: " + str(obs_sequence))

forward_probs = hmm.forward(obs_sequence)
print("\nProbabilidades Forward:")
for t, probs in enumerate(forward_probs):
    print("Tiempo " + str(t) + ": " + str(probs))

backward_probs = hmm.backward(obs_sequence)
print("\nProbabilidades Backward:")
for t, probs in enumerate(backward_probs):
    print("Tiempo " + str(t) + ": " + str(probs))

state_probs = hmm.compute_state_probabilities(obs_sequence)
print("\nProbabilidades de Estado (Posteriores):")
for t, probs in enumerate(state_probs):
    print("Tiempo " + str(t) + ": " + str(probs))


Secuencia Generada: ['Rainy', 'Rainy', 'Rainy', 'Sunny', 'Sunny']

Probabilidades Forward:
Tiempo 0: {'Sunny': 0.1, 'Rainy': 0.35}
Tiempo 1: {'Sunny': 0.044000000000000004, 'Rainy': 0.16099999999999998}
Tiempo 2: {'Sunny': 0.01992, 'Rainy': 0.07377999999999998}
Tiempo 3: {'Sunny': 0.036358400000000006, 'Rainy': 0.014475599999999996}
Tiempo 4: {'Sunny': 0.027901568000000005, 'Rainy': 0.004787112}

Probabilidades Backward:
Tiempo 0: {'Sunny': 0.04308720000000002, 'Rainy': 0.08108560000000002}
Tiempo 1: {'Sunny': 0.12044000000000005, 'Rainy': 0.17012000000000005}
Tiempo 2: {'Sunny': 0.4780000000000002, 'Rainy': 0.31400000000000006}
Tiempo 3: {'Sunny': 0.7000000000000002, 'Rainy': 0.5}
Tiempo 4: {'Sunny': 1, 'Rainy': 1}

Probabilidades de Estado (Posteriores):
Tiempo 0: {'Sunny': 0.13181076751952053, 'Rainy': 0.8681892324804794}
Tiempo 1: {'Sunny': 0.1621160597491242, 'Rainy': 0.8378839402508756}
Tiempo 2: {'Sunny': 0.29128615777694306, 'Rainy': 0.708713842223057}
Tiempo 3: {'Sunny': 0.778

Tras aplicar el algoritmo Forward-Backward sobre la secuencia generada ['Rainy', 'Rainy', 'Sunny', 'Rainy', 'Sunny'], se observaron las probabilidades posteriores de estar en cada estado en cada tiempo. Inicialmente, se tiene alta probabilidad de que el estado real haya sido "Rainy", dado que las primeras observaciones fueron lluviosas. Conforme avanzan los días y cambian las observaciones, las probabilidades reflejan que es más probable que el estado sea "Sunny" en los días donde se observa sol. En general, las probabilidades calculadas permiten inferir el estado real oculto a partir de las observaciones visibles, integrando la información tanto pasada como futura.

| Tiempo | Observación | Estado más probable | Probabilidad |
|:------:|:-----------:|:-------------------:|:------------:|
|   0    | Rainy       | Rainy                | 84.3%        |
|   1    | Rainy       | Rainy                | 74.6%        |
|   2    | Sunny       | Sunny                | 63.3%        |
|   3    | Rainy       | Rainy                | 53.3%        |
|   4    | Sunny       | Sunny                | 76.8%        |


Referencias: 

 -  Rabiner, L. R. (1989).A Tutorial on Hidden Markov Models and Selected Applications in Speech Recognition.Disponible en: https://www.cs.ubc.ca/~murphyk/Bayes/rabiner.pdf​

 -  Ghahramani, Z., & Jordan, M. I. (1997). Factorial Hidden Markov Models.Disponible en: https://mlg.eng.cam.ac.uk/zoubin/papers/fhmmML.pdf​

 -  Durbin, R., Eddy, S. R., Krogh, A., & Mitchison, G. J. (1998). Biological Sequence Analysis: Probabilistic Models of Proteins and Nucleic Acids.

